In [ ]:
import shutil
import os

src_dir = './data/all'
train_dest_dir = './data/train'
val_dest_dir = './data/val'
test_dest_dir = './data/eval'

if len(os.listdir(train_dest_dir)) >= 1:
    # return flac files into original folder if there are any
    print(f'found {len(os.listdir(train_dest_dir))
                    } flac file(s) in train directory...')
    print(f'transferring flac files back to train directory...')
    for file in os.listdir(train_dest_dir):
        shutil.move(f"{train_dest_dir}/{file}", f"{src_dir}/{file}")

if len(os.listdir(val_dest_dir)) >= 1:
    # return flac files into original folder if there are any
    print(f'found {len(os.listdir(val_dest_dir))
                    } flac file(s) in val directory...')
    print(f'transferring flac files back to train directory...')
    for file in os.listdir(val_dest_dir):
        shutil.move(f"{val_dest_dir}/{file}", f"{src_dir}/{file}")

if len(os.listdir(test_dest_dir)) >= 1:
    # return flac files into original folder if there are any
    print(f'found {len(os.listdir(test_dest_dir))
                    } flac file(s) in val directory...')
    print(f'transferring flac files back to train directory...')
    for file in os.listdir(test_dest_dir):
        shutil.move(f"{test_dest_dir}/{file}", f"{src_dir}/{file}")


In [ ]:
import pandas as pd
import shutil

df = pd.read_csv('./data/all_labels.txt', sep=' ', header=None)
df.rename(columns={1: 'key', 5: 'label', 7: 'group'}, inplace=True)
df = df[['key', 'label', 'group']]

for file in df[df['label'] == 'bonafide']['key']:
    shutil.move(f"./data/all/{file}.flac", f"./data/all_bonafide/{file}.flac")
    
for file in df[df['label'] == 'spoof']['key']:
    shutil.move(f"./data/all/{file}.flac", f"./data/all_spoof/{file}.flac")

In [ ]:
import librosa
import numpy as np
from models import SimpleCNN_STFT_FRAMESIZE_1024, FOURTH_CNN_STFT_FRAMESIZE_1024
import torch
import math

from scripts import predict_single_audio
from data_utils import split_audio, preprocess_audio_for_cnn

In [ ]:

    
# read in audio
test_file = 'test_audio.mp3'
data_dir = './data/'
file_path = f'{data_dir}/{test_file}'
SAMPLE_RATE = 16000
# ../data/trial/DF_E_3521558.flac


audio, _ = librosa.load(file_path, sr=SAMPLE_RATE)

print(audio.shape[0])
#initialize model
# model = SimpleCNN_STFT_FRAMESIZE_1024()
model = SimpleCNN_STFT_FRAMESIZE_1024()
# model.load_state_dict(torch.load('./models/FOURTH_CNN_STFT_FRAMESIZE_1024_2SEC_1_weights.pt', weights_only=True))

y_probs = predict_single_audio(audio, model)

# SAMPLE_RATE = 16000
# # split audio into ~2s chunks
# sample_size = 32000
# max_audio_size = audio.shape[0]
# num_secs = math.ceil(max_audio_size/SAMPLE_RATE)

# audio_splits = []
# for start_range in range(0, max_audio_size, sample_size):
#     audio_split = split_audio(
#         audio, start_range, sample_size, max_audio_size)
#     audio_splits.append(audio_split)

# device = "cuda" if torch.cuda.is_available() else "cpu"
# if device == "cpu":
#     raise ValueError("GPU not detected!")

# model = model.to(device)

# y_probs = []
# for X_raw in audio_splits:

#     X = preprocess_audio_for_cnn(X_raw)
#     break
    
#     X = X.to(device)
#     y = model(X)
#     y_probs.extend(np.repeat(y.detach().cpu().numpy()[0][1], 2).tolist())

# print(len(y_probs))
# y_probs = np.array(y_probs[0:num_secs])
# print(len(y_probs))
    
# if audio is 2.5 seconds -> ground truth should be an array of size 3
# if audio is exactly 2 seconds ->ground truth should be an array of size 2
# if audio is 1.99 seconds ->ground truth should be an array of size 2


In [ ]:
y_probs

In [1]:
from data_utils import create_dataloader, Dataset_ASVspoof2021_Raw, Dataset_ASVspoof2021_STFT

batch_size = 12
train_labels_path = './data/trial_labels.txt'
train_data_path = './data/trial'
train_loader = create_dataloader(
    Dataset_ASVspoof2021_STFT, train_labels_path, train_data_path, 64000, batch_size)

no. files in  dataloader: 77


In [2]:
for batch, (X_train, y_train) in enumerate(train_loader):
    break

X_train.shape

c:\Users\jieqi\Desktop\JQ\code_projects\audio_deepfake_detector\data_utils.py:61: UserWarning: PySoundFile failed. Trying audioread instead.
  X_raw, _ = librosa.load(
c:\Users\jieqi\Desktop\JQ\code_projects\audio_deepfake_detector\venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


torch.Size([12, 3, 513, 251])

In [3]:
from models import AASISTModel, SimpleCNN_STFT_FRAMESIZE_1024
import torch

# "nb_samp": 64600,
d_args = {"first_conv": 128,
          "filts": [70, [1, 32], [32, 32], [32, 64], [64, 64]],
          "gat_dims": [64, 32],
          "pool_ratios": [0.5, 0.7, 0.5, 0.5],
          "temperatures": [2.0, 2.0, 100.0, 100.0]
          }

# model = AASISTModel(d_args)
model = SimpleCNN_STFT_FRAMESIZE_1024()
total_params = sum(p.numel() for p in model.parameters())
model = model.to('cuda')

print(f'total_params: {total_params}')
X_train = X_train.to('cuda')
# hidden, y_pred = model(X_train, Freq_aug=False)

X_train = X_train.to('cuda')
y_pred = model(X_train)

# print(hidden.shape)
print(y_pred.shape)


total_params: 8831586
torch.Size([12, 2])


In [ ]:
start_time = time.time()

In [ ]:
df[df['group'] == 'hidden'].groupby('label').count()

34560 * 0.01



In [ ]:
import pandas as pd
df = pd.read_csv('./data/all_labels.txt',sep=' ', header=None)
df.rename(columns={1: 'key', 5: 'label', 7: 'group'},inplace=True)
df = df[['key','label','group']]



In [ ]:
from data_utils import split_audio_dataset

split_audio_dataset()

In [ ]:
import pandas as pd
train_labels_path = './data/train_labels.txt'
train_labels = pd.read_csv(
    train_labels_path, sep=' ', names=['key', 'label'])
train_labels

In [ ]:
from librosa.feature import delta
import numpy as np
import librosa
import torch

SAMPLE_RATE = 16000
FRAME_SIZE = 1024
HOP_SIZE = int(FRAME_SIZE/4)

base_dir = './data/trial'
key = 'DF_E_2010009'
cut = 32300

def pad_random(x: np.ndarray, max_len: int = 64600):
    x_len = x.shape[0]
    # if duration is already long enough
    if x_len >= max_len:
        stt = np.random.randint(x_len - max_len)
        return x[stt:stt + max_len]

    # if too short
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (num_repeats))[:max_len]
    return padded_x



X_raw, _ = librosa.load(
    str(f"{base_dir}/{key}.flac"), sr=SAMPLE_RATE)
X_pad = pad_random(X_raw, cut)
X_stft = librosa.stft(y=X_pad, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
X_log_stft = librosa.power_to_db(np.abs(X_stft)**2)
X_delta = librosa.feature.delta(X_log_stft, width=9, order=1)
X_delta2 = librosa.feature.delta(X_log_stft, width=9, order=2)
stacked = [arr.reshape((1, X_log_stft.shape[0], X_log_stft.shape[1]))
           for arr in (X_log_stft, X_delta, X_delta2)]
X = torch.FloatTensor(np.concatenate(stacked, axis=0))
print(X.shape)

In [ ]:
import torch
test = torch.Tensor(1)


In [ ]:
test.item()

In [ ]:
def get_audio_log_stft(audio):
    
    audio_stft = librosa.stft(y=audio, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    return librosa.power_to_db(audio_stft)
    

# Approach 1: STFT + CNN

In [ ]:
audio, sr = librosa.load(f'{data_path}/{files[0]}',sr=SAMPLE_RATE)


# Approach 2: Raw Waveform Wav2Vec2

# Approach 3: Raw Waveform AAsist

# Approach 4: Raw Waveform Wav2Vec2 + AAsist

In [ ]:
from data_utils import split_audio_dataset

split_audio_dataset()

In [ ]:




# stft with librosa


# feed processed waveform to front_end_processor

inputs = front_end_processor(audio_log_stft, sampling_rate=sr, return_tensors="pt")

# feed frequency into
audio_cqt.shape

In [ ]:
import numpy as np
np.random.randint(0)

In [ ]:
import torch
max_len = 5

stt = 0
x = torch.Tensor([1,2,3,4,5])
x[stt:stt + max_len]

In [ ]:
from data_utils import split_audio_dataset

train_labels_file = './data/train_labels.txt'
train_dir = './data/train'
val_labels_file = './data/val_labels.txt'
val_dir = './data/val'


split_audio_dataset(train_labels_file, train_dir, val_labels_file, val_dir)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(columns=['model','iteration', 'gpu', 'num_params', 'training_time', 'min_train_loss', 'min_val_loss', 'max_train_acc', 'max_val_acc', 'threshold', 'f1', 'recall', 'precision', 'eer'])


In [ ]:
df.to_csv('./results/run_summary.csv',index=False)

In [ ]:
import torch
    
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))
if device == "cpu":
    raise ValueError("GPU not detected!")

In [ ]:
torch.cuda.get_device_name()

In [ ]:
df = pd.DataFrame(columns=['col_1','col_2'])

In [ ]:
data = {'col_1': 'asdas', 'col_2': 2}
pd.concat([df,pd.DataFrame.from_dict(data,orient='index').T],ignore_index=True)

In [ ]:
import torch

batch_size = 3

my_tensor = torch.FloatTensor([[1,2],
                               [3,4],
                               [5,6]])


torch.masked_select(my_tensor, torch.BoolTensor([False,True]).repeat(batch_size,1))




In [ ]:
my_tensor.shape

In [ ]:
torch.Tensor([1,1,1,1,1]).view(-1)

In [ ]:
correct_in_epoch = 0
predicted = torch.Tensor([0,1,1,1])
y_truth = torch.Tensor([0,0,0,1])
correct_in_epoch += (predicted == y_truth).sum()
print(correct_in_epoch)
    

In [ ]:
from models import SimpleCNN_STFT_FRAMESIZE_1024

model = SimpleCNN_STFT_FRAMESIZE_1024()

for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

In [ ]:
import os

len(os.listdir('./data/train'))/24

In [ ]:
import os
import shutil

src_dir = './data/trial'
destination_dir = './data/all'

for file in os.listdir(src_dir):
    shutil.move(f"{src_dir}/{file}", f"{destination_dir}/{file}")

In [ ]:
import os
import shutil

df = pd.read_csv('./data/trial_labels.txt',sep=' ', names=['key','label'])

src_dir = './data/all'
destination_dir = './data/trial'

for file in df['key']:
    shutil.copy(f"{src_dir}/{file}.flac", f"{destination_dir}/{file}.flac")
    

In [ ]:
torch.masked_select(y_val, torch.BoolTensor([False,True]).repeat(y_val.shape[0],1).to("cuda"))

In [ ]:
import numpy as np


x = np.array([[1,2],[3,4],[5,6]])
mask = np.repeat([[False,True]],3,axis=0)
x[mask]


In [ ]:
y_truth = [0,0]
probs = np.array([[0.4,0.6],[0.6,0.4]])
preds = np.argmax(probs,axis=1)
total_correct_in_epoch = sum(preds == y_truth)
print(total_correct_in_epoch)




In [ ]:
import os
import pandas as pd
import shutil

src_dir = './data/train'
destination_dir = './data/trial'

df = pd.read_csv('./data/trial_labels.txt',header=None,sep=' ')
df.rename(columns={0:'key',1:'label'},inplace=True)

for key in df['key']:
    shutil.copyfile(f'{src_dir}/{key}.flac',f'{destination_dir}/{key}.flac')


In [ ]:

df

In [ ]:
import pandas as pd

base_summary = pd.DataFrame(columns=['a', 'b', 'c', 'd'])
summary_stats = {'a': 1, 'b': 2, 'c': 3, 'd': 4}


final_summary = pd.concat([base_summary, pd.DataFrame.from_dict(
    summary_stats, orient='index').T], ignore_index=True)
final_summary = final_summary.fillna(0)
final_summary

In [ ]:
import os
import random
import librosa
import numpy as np

def pad_random(x: np.ndarray, max_len: int = 64600):
    x_len = x.shape[0]
    # if duration is already long enough
    if x_len > max_len:
        stt = np.random.randint(x_len - max_len)
        return x[stt:stt + max_len]
    elif x_len == max_len:
        return x

    # if too short
    num_repeats = int(max_len / x_len) + 1
    padded_x = np.tile(x, (num_repeats))[:max_len]
    return padded_x


def calculate_audio_mean_std(data_dir, sample_size):
    audio_file_list = os.listdir(data_dir)
    start = random.randint(0, len(audio_file_list) - 1 - sample_size)

    raw_data = []
    delta_data = []
    delta2_data = []

    for file in audio_file_list[start:start+sample_size]:
        SAMPLE_RATE = 16000
        FRAME_SIZE = 1024
        HOP_SIZE = int(FRAME_SIZE/4)
        
        X_raw = librosa.load(f"{data_dir}/{file}", sr=SAMPLE_RATE)[0]
        X_pad = pad_random(X_raw, 32000)
        X_stft = librosa.stft(y=X_pad, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
        X_log_stft = librosa.power_to_db(np.abs(X_stft)**2)
        raw_data.extend(X_log_stft.flatten().tolist())
        X_delta = librosa.feature.delta(X_log_stft, width=9, order=1)
        delta_data.extend(X_delta.flatten().tolist())
        X_delta2 = librosa.feature.delta(X_log_stft, width=9, order=2)
        delta2_data.extend(X_delta2.flatten().tolist())

        # stacked = [arr.reshape((1, X_log_stft.shape[0], X_log_stft.shape[1]))
        #            for arr in (X_log_stft, X_delta, X_delta2)]
        # X = torch.FloatTensor(np.concatenate(stacked, axis=0))
    raw_mean = np.array(raw_data).mean()
    raw_std = np.array(raw_data).std()
    delta_mean = np.array(delta_data).mean()
    delta_std = np.array(delta_data).std()
    delta2_mean = np.array(delta2_data).mean()
    delta2_std = np.array(delta2_data).std()

    return [(raw_mean, raw_std), (delta_mean, delta_std), (delta2_mean, delta2_std)]


sample_size = 2000
data_dir = './data/train'

raw_stats, delta_stats, delta2_stats = calculate_audio_mean_std(
    data_dir, sample_size)

In [ ]:
raw_stats, delta_stats, delta2_stats